This notebook uses data from each participant individually. A spiking neural network is created for every participant
and then tested on 25% of the participant's data which were previously not used for training.

In [10]:
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
import nengo
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder

In [4]:
dataset_path = os.path.join('dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1))
         for i in range(18)] # P01 - P18 files

In [5]:
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

First we check data for the first file,
these operations are later applied for every file of the dataset

In [7]:
# Check data in the first file then apply it for every file
dataset = np.load(files[0])
features, labels = dataset['features'], dataset['labels']

print(features.shape)
print(labels.shape)

(120, 14, 36, 10)
(120,)


In [15]:
print(labels[50:53]) # no, yes, no

['no' 'yes' 'no']


In [19]:
cat = OneHotEncoder()
labels = labels.reshape(-1, 1)
print(labels[50:53])
labels = cat.fit_transform(labels).toarray()
print(labels[50:53])
labels = labels.reshape((labels.shape[0], 1, -1)) # add time dimension for nengo
print(labels[50:53])

[[0.]
 [1.]
 [1.]]
[[1. 0.]
 [0. 1.]
 [0. 1.]]
[[[1. 0.]]

 [[0. 1.]]

 [[0. 1.]]]


Now define tensorflow models which will be tested


In [ ]:
def cnn_model_1():
    """
    Creates a CNN neural network
    :return: tensorflow model of the ANN
    """

    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv2d = Conv2D(filters=9, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.5, seed=seed)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(2, 2))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(1000, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(500, activation='relu')(batch_norm)
    dropout2 = Dropout(0.5, seed=seed)(dense2)
    output = Dense(2, activation='softmax', name='output_layer')(dropout2)

    return Model(inputs=inp, outputs=output)

In [1]:
def train_model(model, x_train, y_train, params_output_path, num_epochs=10):
    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=4) as simulator:
        simulator.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        simulator.fit(
            x={ input_layer: x_train },
            y={ output_layer: y_train },
            epochs=num_epochs,
            shuffle=True
        )

        simulator.save_params(params_output_path)

In [ ]:
def test_model(model, params_path, x_test, y_test, scale_firing_rates=1000, synapse=0.01, timesteps=30):

    converter = nengo_dl.Converter(model)
    with nengo_dl.Simulator(converter.net, minibatch_size=4) as simulator:
        simulator.load_params(params_path)

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        ann_eval = simulator.evaluate(
            x={ input_layer: x_test },
            y={ output_layer: y_test }
        ) # results from ann

    # Converter for nengo spiking network
    converter = nengo_dl.Converter(
        model,
        swap_activations={ tf.nn.relu: nengo.SpikingRectifiedLinear() },
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    )

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    input_layer = converter.inputs[model.get_layer('input_layer')] # input layer for simulator
    output_layer = converter.outputs[model.get_layer('output_layer')] # output layer for simulator

    x_test_tiled = np.tile(x_test, (1, timesteps, 1))

    with nengo_dl.Simulator(converter.net, minibatch_size=4) as simulator:
        simulator.load_params(params_path)

        predictions = simulator.predict({ input_layer: x_test_tiled })[output_layer]
        predictions = predictions[:, -1, :] # get last timestep from prediction
        y_test = np.squeeze(y_test, axis=1)
        y_test = np.argmax(y_test, axis=-1)

        snn_acc = (predictions == y_test).mean()

    return ann_eval, snn_acc